# project : song Lyrics 


## data download

In [3]:
#!wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip


--2020-09-08 16:52:58--  https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
Resolving aiffelstaticprd.blob.core.windows.net (aiffelstaticprd.blob.core.windows.net)... 52.239.148.4
접속 aiffelstaticprd.blob.core.windows.net (aiffelstaticprd.blob.core.windows.net)|52.239.148.4|:443... 접속됨.
HTTP request sent, awaiting response... 200 OK
Length: 2101791 (2.0M) [application/zip]
Saving to: ‘song_lyrics.zip’

song_lyrics.zip     100%[===================>]   2.00M  1.86MB/s    in 1.1s    

2020-09-08 16:53:00 (1.86 MB/s) - ‘song_lyrics.zip’ saved [2101791/2101791]

/bin/bash: uzip: 명령어를 찾을 수 없음


## library 

In [1]:
import glob
import os

import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
from sklearn.model_selection import train_test_split



## data import 


In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/AIFFEL_LSG/lms/01-EXPLORATION/E11-language_model/data/song_lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['The Cat in the Hat', 'By Dr. Seuss', 'The sun did not shine.']


## data 정제

In [3]:
### 

In [4]:
# 정규표현식 활용
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> the cat in the hat <end>',
 '<start> by dr . seuss <end>',
 '<start> the sun did not shine . <end>',
 '<start> it was too wet to play . <end>',
 '<start> so we sat in the house <end>',
 '<start> all that cold cold wet day . <end>',
 '<start> i sat there with sally . <end>',
 '<start> we sat there we two . <end>',
 '<start> and i said how i wish <end>',
 '<start> we had something to do ! <end>']

In [6]:
#tf keras txt tokenizer
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15) 

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    6  903 ...    0    0    0]
 [   2  122 2584 ...    0    0    0]
 [   2    6  304 ...    0    0    0]
 ...
 [ 673   27    6 ...    6  189    3]
 [   2  673   27 ...    0    0    0]
 [   2  673   27 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f27a8354d10>


In [7]:
print(tensor[:3, :10])

[[   2    6  903   14    6 1350    3    0    0    0]
 [   2  122 2584   20    1    3    0    0    0    0]
 [   2    6  304  166   70  559   20    3    0    0]]


In [8]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [26]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### ????????????????

In [14]:

# tensors = []

# for i in range(len(tensor)):
#     if tensor[i][1] == tensor[i][2]: continue
#     if tensor[i][2] == tensor[i][3]: continue
#     tensors.append(tensor[i,:])

# print(tensors[0])

[   2    6  903   14    6 1350    3    0    0    0    0    0    0    0
    0]


In [15]:
tensors = np.array(tensors)

### 소스문장 타켓문장 생성

In [19]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2    6  903   14    6 1350    3    0    0    0    0    0    0    0]
[   6  903   14    6 1350    3    0    0    0    0    0    0    0    0]


## 평가데이터셋 분리
- tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!



In [20]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

In [21]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


out:

Source Train: (124960, 14)   
Target Train: (124960, 14)   

## 인공지능 만들기
- 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

In [22]:
tokenizer.num_words

12000

### model

In [23]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [27]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.3199903e-04,  2.9871066e-04,  4.9130249e-05, ...,
         -1.5502221e-04,  2.1695627e-05, -1.8319429e-04],
        [ 2.4767886e-04,  4.2349991e-04,  2.7915981e-04, ...,
         -3.7004409e-04,  1.0434317e-06, -2.9595874e-04],
        [ 4.3099205e-04,  6.6848111e-04,  5.0609163e-04, ...,
         -4.3093206e-04,  8.5963416e-05, -4.8015080e-04],
        ...,
        [ 2.6365180e-05, -1.6809209e-03,  1.8648523e-03, ...,
          6.0892315e-04,  1.4180929e-03, -1.4180611e-03],
        [-2.3654150e-04, -1.8829730e-03,  1.9569884e-03, ...,
          7.4189540e-04,  1.6371636e-03, -1.5288084e-03],
        [-4.9706950e-04, -1.9820037e-03,  2.0157106e-03, ...,
          8.7420532e-04,  1.8487130e-03, -1.5979615e-03]],

       [[ 1.3199903e-04,  2.9871066e-04,  4.9130249e-05, ...,
         -1.5502221e-04,  2.1695627e-05, -1.8319429e-04],
        [ 2.1680405e-04,  7.6040736e-04,  3.1068514e-05, ...,
          1.2495695e-04, 

In [28]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


### loss & optimizer

In [29]:
#Loss

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

optimizer = tf.keras.optimizers.Adam()

### 훈련

In [31]:
model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10)

Epoch 1/10
4394/4394 [==============================] - 155s 35ms/step - loss: 3.3282
Epoch 2/10
4394/4394 [==============================] - 156s 35ms/step - loss: 2.8995
Epoch 3/10
4394/4394 [==============================] - 155s 35ms/step - loss: 2.6284
Epoch 4/10
4394/4394 [==============================] - 156s 36ms/step - loss: 2.3922
Epoch 5/10
4394/4394 [==============================] - 156s 36ms/step - loss: 2.1840
Epoch 6/10
4394/4394 [==============================] - 156s 36ms/step - loss: 2.0002
Epoch 7/10
4394/4394 [==============================] - 157s 36ms/step - loss: 1.8361
Epoch 8/10
4394/4394 [==============================] - 156s 36ms/step - loss: 1.6892
Epoch 9/10
4394/4394 [==============================] - 157s 36ms/step - loss: 1.5572
Epoch 10/10
4394/4394 [==============================] - 156s 36ms/step - loss: 1.4409


## 평가

In [32]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.



In [34]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [36]:
generate_text(model, tokenizer, init_sentence="<start> i am", max_len=20)

'<start> i am the walrus <end> '

In [37]:
generate_text(model, tokenizer, init_sentence="<start> show me the ", max_len=20)

'<start> show me the target so i can t wait to fight <end> '

In [39]:
generate_text(model, tokenizer, init_sentence="<start> i hate ", max_len=20)

'<start> i hate the way you lie <end> '